In [ ]:
%pip install google-cloud-storage

In [ ]:
import os
import json
import tempfile
from google.cloud import storage
from tqdm import tqdm


# === 1. Your service account credentials (pasted JSON) ===
service_account_json = {}

# === 2. Configuration ===
bucket_name = "models-bucket-munshi"
folders_to_upload = ["./phi4-full-finetune-2", "./phi4-full-finetune"]
destination_base_prefix = "models"  # top-level folder in GCS bucket

# === 3. Write service account key to a temp file ===
with tempfile.NamedTemporaryFile("w+", suffix=".json", delete=False) as tmp_key_file:
    json.dump(service_account_json, tmp_key_file)
    tmp_key_path = tmp_key_file.name

# === 4. Initialize GCS client ===
client = storage.Client.from_service_account_json(tmp_key_path)
bucket = client.get_bucket(bucket_name)


# === 5. Upload with progress
def upload_with_progress(blob, file_path):
    file_size = os.path.getsize(file_path)
    with open(file_path, "rb") as f:
        with tqdm.wrapattr(f, "read", total=file_size, desc=f"Uploading {os.path.basename(file_path)}", unit="B", unit_scale=True, unit_divisor=1024) as file_obj:
            blob.upload_from_file(file_obj, rewind=True)

# === 6. Upload folders
for folder_path in folders_to_upload:
    model_name = os.path.basename(folder_path.rstrip("/"))
    destination_prefix = os.path.join(destination_base_prefix, model_name)

    for root, _, files in os.walk(folder_path):
        for file in files:
            local_path = os.path.join(root, file)
            relative_path = os.path.relpath(local_path, folder_path)
            gcs_path = os.path.join(destination_prefix, relative_path)

            blob = bucket.blob(gcs_path)
            upload_with_progress(blob, local_path)

print("✅ All model folders uploaded with progress tracking.")
os.remove(tmp_key_path)